In [1]:
import os
import re
import sys
import torch
import hashlib
import itertools
import logging
import numpy as np

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


from progress.bar import Bar
from concurrent.futures import ProcessPoolExecutor

from Utils.NotesSeq import NoteSeq as ns
from Utils.EventSeq import EventSeq as es
from Utils.ControlSeq import ControlSeq as cs
from Utils import utils

import warnings
warnings.filterwarnings("ignore")

In [2]:
from pretty_midi import PrettyMIDI, Note, Instrument

In [3]:
def preprocess_midi(path):

    note_seq = ns.from_midi_file(path)
    iter = itertools.cycle(note_seq)

    es1 = list
    es2 = list

    cs1 = list
    cs2 = list
    ev = True
    #print("evaluating")
    for seq in iter:
        #print(seq)
        if ev:
            #print("1st iter")
            seq.adjust_time(-seq.notes[0].start)
            event_seq = es.from_note_seq(seq)
            control_seq = cs.from_event_seq(event_seq)
            es1 = event_seq.to_array()
            cs1 = control_seq.to_compressed_array()
            ev = False
        else:
            #print("2nd iter")
            seq.adjust_time(-seq.notes[0].start)
            event_seq = es.from_note_seq(seq)
            control_seq = cs.from_event_seq(event_seq)
            es2 = event_seq.to_array()
            cs2 = control_seq.to_compressed_array()
            break

    return es1, es2, cs1, cs2

In [4]:
midi_path = './Naruto_Shippuden_-_Silhouette.mid'

In [5]:
def from_tracks(midi, flag, programs=range(128)):
    group_indices = flag # Variable flag permits to manage different tracks
    group_notes = itertools.chain(*[
        midi.instruments[i].notes for i in group_indices
    ])
    #print(list(group_notes))
    #print(list(group_notes))

    temp = list(group_notes)
    #print(temp)
    #print(70*'-')
    for note in temp:
        assert isinstance(note, Note)
    notes = list(filter(lambda note: note.end >= note.start, temp)) # filtro per essere sicuri che la nota estratta sia qurlla corretta
    #notes = notes.sort(key=lambda note: note.start)
    #print(notes)

    return ns(list(notes))

In [6]:
def from_midi_file(path):
        midi = PrettyMIDI(path)
        track_len = len(midi.instruments)
        print(midi.instruments)
        if(track_len < 3):
            stream1 = from_tracks(midi,[0])
            print(stream1.notes)
            stream2 = from_tracks(midi,[1])
            #print(" 2 tracce : " + path)
        elif(track_len < 4):
            stream1 = from_tracks(midi,[0])
            stream2 = from_tracks(midi,[1,2])
            #print(" 3 tracce : " + path)
        else:
            stream1 = from_tracks(midi,[0,1])
            stream2 = from_tracks(midi,[2,3])
            #print(" 4 tracce : " + path)

        return stream1,stream2

In [7]:
midi = PrettyMIDI(midi_path)
track_len = len(midi.instruments)

In [8]:
midi.instruments

[Instrument(program=0, is_drum=False, name=""),
 Instrument(program=0, is_drum=False, name="")]

In [9]:
s1,s2 = from_midi_file(midi_path)

[Instrument(program=0, is_drum=False, name=""), Instrument(program=0, is_drum=False, name="")]
[Note(start=0.000000, end=0.153379, pitch=59, velocity=112), Note(start=0.162163, end=0.315541, pitch=57, velocity=112), Note(start=0.324325, end=0.477704, pitch=59, velocity=112), Note(start=0.486488, end=0.793921, pitch=62, velocity=112), Note(start=0.810813, end=0.964191, pitch=57, velocity=112), Note(start=0.972975, end=1.126354, pitch=59, velocity=112), Note(start=1.135138, end=1.288516, pitch=57, velocity=112), Note(start=1.297300, end=1.450679, pitch=59, velocity=112), Note(start=1.459463, end=1.766896, pitch=62, velocity=112), Note(start=1.783788, end=1.937166, pitch=57, velocity=112), Note(start=1.945950, end=2.253383, pitch=59, velocity=112), Note(start=2.594600, end=3.056087, pitch=69, velocity=112), Note(start=2.594600, end=3.056087, pitch=74, velocity=112), Note(start=3.081088, end=3.542575, pitch=69, velocity=112), Note(start=3.567575, end=3.875008, pitch=74, velocity=112), Note

In [10]:
print(s2.notes)

[Note(start=0.000000, end=0.153379, pitch=35, velocity=96), Note(start=0.000000, end=0.153379, pitch=47, velocity=96), Note(start=0.162163, end=0.315541, pitch=35, velocity=96), Note(start=0.162163, end=0.315541, pitch=47, velocity=96), Note(start=0.324325, end=0.477704, pitch=35, velocity=96), Note(start=0.324325, end=0.477704, pitch=47, velocity=96), Note(start=0.486488, end=0.639866, pitch=33, velocity=96), Note(start=0.486488, end=0.639866, pitch=45, velocity=96), Note(start=0.648650, end=0.802029, pitch=33, velocity=96), Note(start=0.648650, end=0.802029, pitch=45, velocity=96), Note(start=0.810813, end=0.964191, pitch=33, velocity=96), Note(start=0.810813, end=0.964191, pitch=45, velocity=96), Note(start=0.972975, end=1.126354, pitch=33, velocity=96), Note(start=0.972975, end=1.126354, pitch=45, velocity=96), Note(start=1.135138, end=1.288516, pitch=33, velocity=96), Note(start=1.135138, end=1.288516, pitch=45, velocity=96), Note(start=1.297300, end=1.450679, pitch=31, velocity=9

In [11]:
s1.adjust_time(-s1.notes[0].start)

In [12]:
USE_VELOCITY = True
DEFAULT_TEMPO = 120
BEAT_LENGTH = 60 / DEFAULT_TEMPO
DEFAULT_TIME_SHIFT_BINS = 1.15 ** np.arange(32) / 65
DEFAULT_VELOCITY_STEPS = 32
DEFAULT_NOTE_LENGTH = BEAT_LENGTH * 2
MIN_NOTE_LENGTH = BEAT_LENGTH / 2
DEFAULT_PITCH_RANGE = range(21, 109)
DEFAULT_VELOCITY_RANGE = range(21, 109)

In [13]:
pitch_range = DEFAULT_PITCH_RANGE
velocity_range = DEFAULT_VELOCITY_RANGE
velocity_steps = DEFAULT_VELOCITY_STEPS
time_shift_bins = DEFAULT_TIME_SHIFT_BINS

In [14]:
BEAT_LENGTH

0.5

In [15]:
time_shift_bins

array([0.01538462, 0.01769231, 0.02034615, 0.02339808, 0.02690779,
       0.03094396, 0.03558555, 0.04092338, 0.04706189, 0.05412117,
       0.06223935, 0.07157525, 0.08231154, 0.09465827, 0.10885701,
       0.12518556, 0.1439634 , 0.16555791, 0.19039159, 0.21895033,
       0.25179288, 0.28956182, 0.33299609, 0.3829455 , 0.44038733,
       0.50644542, 0.58241224, 0.66977407, 0.77024019, 0.88577621,
       1.01864265, 1.17143904])

In [16]:
MIN_NOTE_LENGTH

0.25

In [17]:
def get_velocity_bins():
        n = es.velocity_range.stop - es.velocity_range.start
        return np.arange(es.velocity_range.start,
                         es.velocity_range.stop,
                         n / (es.velocity_steps - 1))

In [18]:
get_velocity_bins()

array([ 21.        ,  23.83870968,  26.67741935,  29.51612903,
        32.35483871,  35.19354839,  38.03225806,  40.87096774,
        43.70967742,  46.5483871 ,  49.38709677,  52.22580645,
        55.06451613,  57.90322581,  60.74193548,  63.58064516,
        66.41935484,  69.25806452,  72.09677419,  74.93548387,
        77.77419355,  80.61290323,  83.4516129 ,  86.29032258,
        89.12903226,  91.96774194,  94.80645161,  97.64516129,
       100.48387097, 103.32258065, 106.16129032])

In [19]:
class Event:

    def __init__(self, type, time, value):
        self.type = type # can be of four types: 1)velocity 2)note_on 3)note_off 4)time_shift
        self.time = time # indicates the absoulte time where the event starts
        self.value = value # in case of velocity it indicates the velocity bin, in case of pitch the actual pitch

    def __repr__(self):
        return 'Event(type={}, time={}, value={})'.format(
            self.type, self.time, self.value)

In [20]:
def from_note_seq(note_seq):
    note_events = []
    
    velocity_bins = get_velocity_bins()

    for note in note_seq.notes:
        if note.pitch in pitch_range:

            # Regularization of the velocity of a note. We impose the maximum or minimum velocity in 
            # case the original one is out of our bounds
            velocity = note.velocity
            velocity = max(velocity, velocity_range.start)
            velocity = min(velocity, velocity_range.stop - 1)

            velocity_index = np.searchsorted(velocity_bins, velocity)
            note_events.append(Event('velocity', note.start, velocity_index))

            pitch_index = note.pitch - pitch_range.start
            note_events.append(Event('note_on', note.start, pitch_index))
            note_events.append(Event('note_off', note.end, pitch_index))

    note_events.sort(key=lambda event: event.time)  # stable

    #print(note_events)

    events = []

    for i, event in enumerate(note_events):
        events.append(event)
        #print(event)

        if event is note_events[-1]:
            break

        interval = note_events[i + 1].time - event.time
        shift = 0

        while interval - shift >= time_shift_bins[0]:
            index = np.searchsorted(time_shift_bins,
                                    interval - shift, side='right') - 1
            events.append(Event('time_shift', event.time + shift, index))
            shift += time_shift_bins[index]

    return events

In [21]:
pure_events = from_note_seq(s1)

In [22]:
for event in pure_events:
    assert isinstance(event, Event)

In [23]:
print(type(pure_events[0]))

<class '__main__.Event'>


In [24]:
es_1 = es(pure_events)

In [62]:
class Control:

    def __init__(self, pitch_histogram, note_density):
        self.pitch_histogram = pitch_histogram  # list
        self.note_density = note_density  # int

    def __repr__(self):
        return 'Control(pitch_histogram={}, note_density={})'.format(
            self.pitch_histogram, self.note_density)

    def to_array(self):
        feat_dims = cs.feat_dims()
        ndens = np.zeros([feat_dims['note_density']])
        ndens[self.note_density] = 1.  # [dens_dim]
        phist = np.array(self.pitch_histogram)  # [hist_dim]
        return np.concatenate([ndens, phist], 0)  # [dens_dim + hist_dim]


In [64]:
start, end = 0, 0
pitch_count = np.zeros([12])
note_count = 0

controls = []
def _rel_pitch(pitch):
            return (pitch - 24) % 12

events = list(es_1.events)

In [68]:
from IPython.display import clear_output

In [75]:
for i, event in enumerate(events):
    #print(i)
    #print(event)
    print(f"Start at {start} and Ending at {end}. Cycle {i}/{len(events)}. Note Count {note_count}")

    while start < i:
        if events[start].type == 'note_on':
            abs_pitch = events[start].value + es.pitch_range.start
            rel_pitch = _rel_pitch(abs_pitch)
            pitch_count[rel_pitch] -= 1.
            note_count -= 1.
        start += 1
        print(f"Start Value = {start} Abs pitch = {abs_pitch} Rel pitch = {rel_pitch}")

    while end < len(events):
        if events[end].time - event.time > cs.window_size:
            break
        if events[end].type == 'note_on':
            abs_pitch = events[end].value + es.pitch_range.start
            rel_pitch = _rel_pitch(abs_pitch)
            pitch_count[rel_pitch] += 1.
            note_count += 1.
        end += 1
        print(f"End Value = {end} Abs pitch = {abs_pitch} Rel pitch = {rel_pitch}")
    # list containing the probability of each note (sum = 1)
    pitch_histogram = (
                pitch_count / note_count
                if note_count
                else np.ones([12]) / 12
            ).tolist()
    print(pitch_histogram)

    note_density = max(np.searchsorted(
        cs.note_density_bins,
        note_count, side='right') - 1, 0)
    print(note_density)

    controls.append(Control(pitch_histogram, note_density))

Start at 2209 and Ending at 2210. Cycle 0/2210. Note Count 0.0
[0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]
0
Start at 2209 and Ending at 2210. Cycle 1/2210. Note Count 0.0
[0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]
0
Start at 2209 and Ending at 2210. Cycle 2/2210. Note Count 0.0
[0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]
0
Start at 2209 and Ending at 2210. Cycle 3/2210

In [31]:
def from_event_seq(es):
    events = list(es.events)
    #print(events)
    start, end = 0, 0

    pitch_count = np.zeros([12])
    print(pitch_count)
    note_count = 0

    controls = []

    # Cant understand the utility rn
    def _rel_pitch(pitch):
            return (pitch - 24) % 12
    
    for i, event in enumerate(events):

        while start < i:
            if events[start].type == 'note_on':
                abs_pitch = events[start].value + es.pitch_range.start
                rel_pitch = _rel_pitch(abs_pitch)
                pitch_count[rel_pitch] -= 1.
                note_count -= 1.
            start += 1

        while end < len(events):
            if events[end].time - event.time > cs.window_size:
                break
            if events[end].type == 'note_on':
                abs_pitch = events[end].value + es.pitch_range.start
                rel_pitch = _rel_pitch(abs_pitch)
                pitch_count[rel_pitch] += 1.
                note_count += 1.
            end += 1

    pitch_histogram = (
        pitch_count / note_count
        if note_count
        else np.ones([12]) / 12
    ).tolist()

    note_density = max(np.searchsorted(
        cs.note_density_bins,
        note_count, side='right') - 1, 0)

    controls.append(Control(pitch_histogram, note_density))
    
    return controls


In [32]:
cs_1 = from_event_seq(es_1)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
